# Plotting omics data

In [ ]:
#Load packages

library(tidyverse)
library(plotly)
library(vegan)
library(RColorBrewer)
library(readxl)

#ignore warnings ;)


In [ ]:
#using one of the RColorBrewer can be convenient
#you can see all offered combination
display.brewer.all()

In [ ]:
# and you can set palette you want
palette(brewer.pal(n=12, name = "Set3"))


## Dataset
We will use the amplicon dataset of the bacterial communities of Cariaco Basin (off the coast of Venezuela) from [Suter et al. 2018](https://sfamjournals.onlinelibrary.wiley.com/doi/full/10.1111/1462-2920.13997)
- The Cariaco Basin is a permanently stratified water column. The stratification has created an anoxic and euxinic deep water layer.
- We sampled along the oxygen gradient from the upper oxic layer to bottom anoxic (Oxic -> Dysoxic -> Suboxic -> Anoxic ->Sulfidic->Euxinic) in two different time point (May and November of 2014)

### Import and take a glance at the data

In [ ]:
#Import the output table (after the analysis of sequences)
cariaco_table <- read.delim("input_data/Cariaco_Bac_OTU.txt", sep = "\t")

In [ ]:
#let's verify that it is there
ls()

In [ ]:
#observe the first rows
head(cariaco_table)

In [ ]:
#And the last rows
tail(cariaco_table)

In [ ]:
colSums(cariaco_table[,2:13]==0)

In [ ]:
colSums(cariaco_table[,2:13]==0)/nrow(cariaco_table[,2:13])

In [ ]:
#"Rarecurve" is a plot of the number of species (OTUs) as a function of the number of reads. Rarefaction curves generally grow rapidly at first, as the most common species are found, but the curves plateau as only the rarest species remain to be sampled
#This function requires samples to be the row names and the "species" the columns for this reason we transpose the table
rarecurve(t(cariaco_table[,2:13]),step=1000,col=1:12,xlab = "Reads",ylab = "OTUs",
          lwd=4,cex.lab=0.8,xlim=c(0,1000000),label=F)
legend("topright",rownames(t(cariaco_table[,2:13])),col=1:12,
       cex=0.6,lwd=4,horiz=F, bty="n", inset = c(0, 0.2))

In [ ]:
#Transform the table to long format with pivot_longer
#We will also parse the sample headers. This will add information to our table that will come handy during plotting
#Finally we will separate the taxonomy to it's different ranks ("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")
cariaco_long <- cariaco_table %>%
  pivot_longer(cols = CarOxic_May_103:CarEux_Nov_900, names_to = "Sample", values_to = "Count") %>%
  separate(Sample, c("Feature", "Month", "Depth"), sep = "_", remove = FALSE) %>%
  separate(taxonomy, c("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species"), sep = "; ", remove = FALSE) %>%
  data.frame

In [ ]:
head(cariaco_long)

In [ ]:
dim(cariaco_table)
dim(cariaco_long)

In [ ]:
#check the summaries on the Kindgom level
cariaco_long %>% group_by(Kingdom) %>% summarize (Total_counts=sum(Count))

In [ ]:
#replace "No blast hit" with NA
cariaco_long[cariaco_long =="No blast hit" ] <- NA

In [ ]:
# Summarize the counts to phylum level for each Sample; we will filter out the non-annotated
phylum_sum <- cariaco_long %>%
  filter(!is.na(Kingdom)) %>%
  group_by(Sample, Feature,Month, Phylum) %>%
  summarise(phylum_counts = sum(Count)) %>%
  data.frame

head(phylum_sum)

### Create barplots to start exploring the data 
For most of the plots we are going to use ggplot2. [Datacarpenty has a great tutorial](https://datacarpentry.org/R-ecology-lesson/04-visualization-ggplot2.html) on this package.

In [ ]:
ggplot(phylum_sum, aes(x = Sample, y = phylum_counts, fill = Phylum)) +
  geom_bar(stat = "identity", position = "stack") + theme_bw()+
  theme(axis.text.x = element_text(angle = 90))


In [ ]:
ggplot(phylum_sum, aes(x = Feature, y = phylum_counts, fill = Phylum)) +
  geom_bar(stat = "identity", position = "stack") + facet_grid(Month~.)+ theme_bw()+
  theme(axis.text.x = element_text(angle = 90))

In [ ]:
#there are too many phyla (thus too many colors) and as we can see from the plots we can only discern a handful
#let's try using only "top ten" phyla

phylum10<-cariaco_long %>%
  group_by(Phylum) %>%
  summarise(phylum_counts = sum(Count)) %>% top_n(10)

In [ ]:
#and use the top ten hits to make the list that we are going to use for our selection
phylum10_list<-phylum10$Phylum

In [ ]:
phylum10_list

In [ ]:
#filter the phylum_sum we created above to contain only the phyla found in the top 10 list
phylum10_sum<-phylum_sum %>% filter(Phylum %in% phylum10_list)

In [ ]:
ggplot(phylum10_sum, aes(x = Feature, y = phylum_counts, fill = Phylum)) +
  geom_bar(stat = "identity", position = "stack") + facet_grid(Month~.)+ theme_bw()+
  theme(axis.text.x = element_text(angle = 90), )

In [ ]:
#write your own code
#can check how many reads were excluded by only plotting the 10 more abundant phyla?

In [ ]:
#order and rename the features
#rename the axis
#plot relative abundance
ggplot(phylum10_sum, aes(x = Feature, y = phylum_counts, fill = Phylum)) +
  scale_x_discrete(name ="Oxygen Regime", limits=c("CarOxic", "CarDysox","CarSuboxic","CarAnox","CarSulf", "CarEux"),   labels=c("Oxic",  "Dysoxic", "Suboxic", "Anoxic", "Sulfidic", "Euxinic")) + labs(x='', y = "Relative abundance")+
  geom_bar(stat = "identity", position = "fill") + facet_grid(Month~.)+ theme_bw()+
  theme(axis.text.x = element_text(angle = 45,hjust = 1, vjust = 1), text = element_text(size = 18))


In [ ]:
#write your own code
# can you reverse axes (in order for the y axis to represent depth and go from Oxic (top od water column) to Euxinic (deepest))

In [ ]:
#change the palette, just for fun
ggplot(phylum10_sum, aes(x = Feature, y = phylum_counts, fill = Phylum)) +
  scale_x_discrete(name ="Oxygen Regime", limits=c("CarOxic", "CarDysox","CarSuboxic","CarAnox","CarSulf", "CarEux"),   labels=c("Oxic",  "Dysoxic", "Suboxic", "Anoxic", "Sulfidic", "Euxinic")) + labs(x='', y = "Relative abundance")+
  geom_bar(stat = "identity", position = "fill") + scale_fill_brewer(palette = "Set3") +facet_grid(Month~.)+ theme_bw()+
  theme(axis.text.x = element_text(angle = 45,hjust = 1, vjust = 1),text = element_text(size = 18))

The relative abundance of Proteobacteria seems not to be changing much for the first 5 depths (corresponding to different depths, oxygen etc). Such dramatic changes in the environment should have caused the microbial communities to shift. But, Proteobacteria is a very diverse phylum so the changes with be obvious in the "subgroups" e.g. Class level. Let's explore this further, 

In [ ]:
# Summarize to phylum level, filter out the non-annotated, and select only Proteobacteria. Group by class
class_sum <- cariaco_long %>%
  filter(!is.na(Kingdom)& Phylum=="Proteobacteria") %>%
  group_by(Sample, Feature,Month, Class) %>%
  summarise(class_counts = sum(Count)) %>%
  data.frame

In [ ]:
ggplot(class_sum, aes(x = Feature, y = class_counts, fill = Class)) +
  scale_x_discrete(name ="Oxygen Regime", limits=c("CarOxic", "CarDysox","CarSuboxic","CarAnox","CarSulf", "CarEux"),   labels=c("Oxic",  "Dysoxic", "Suboxic", "Anoxic", "Sulfidic", "Euxinic")) + labs(x='', y = "Relative abundance")+
  geom_bar(stat = "identity", position = "fill") + facet_grid(Month~.)+ theme_bw()+
  theme(axis.text.x = element_text(angle = 45,hjust = 1, vjust = 1), text = element_text(size = 18))

## Ordinations
Cluster analysis encompasses several multivariate techniques that are used to group objects into categories based on their dissimilarities. It is a commonly used method for revealing patterns (and investigate potential effects of environmental factors; see below).  
There are several different methods (each with its own assumptions on the normality etc of the data). The biggest challenge of applying these methods in omics data is the presence of zeros.   
We do not have time to discuss all these aspects but you can find more info in the excellent paper by [Ramette 2017](https://academic.oup.com/femsec/article/62/2/142/434668).
Please also check Sarah's [excellent tutorial](https://academic.oup.com/femsec/article/62/2/142/434668)

The data need [transformation](http://biol09.biol.umontreal.ca/PLcourses/Section_7.7_Transformations.pdf). One of the most commonly used transformations is helling. Log-ratio transformations are also used often. 

In [ ]:
#hellinger transformatipn
carhell <- decostand (t(cariaco_table[,2:13]), method = 'hellinger')

Principal Components Analysis

In [ ]:
#PCA analysis
#remember that you can use ?prcomp to check the command
carhell_pca <- prcomp(carhell)
class(carhell_pca)
carhell_pca$sdev # Explore components of prcomp output

In [ ]:
# Visual representation - screeplot
# This is just a guide to help you decide on a 2-D vs 3-D analysis
# Let's skip due to time constrains
carhell_pca_variances <- as.data.frame(carhell_pca$sdev^2/sum(carhell_pca$sdev^2)) %>% #Extract axes
# Format to plot
select(Var = 'carhell_pca$sdev^2/sum(carhell_pca$sdev^2)') %>%
rownames_to_column(var = "PCaxis") %>%
  data.frame

# Plot screeplot
ggplot(carhell_pca_variances, aes(x = as.numeric(PCaxis), y = Var)) + 
  geom_bar(stat = "identity", fill = "grey", color = "black") +
  theme_minimal() +
  theme(axis.title = element_text(color = "black", face = "bold", size = 10),
        axis.text.y = element_text(color = "black", face = "bold"),
        axis.text.x = element_blank()) +
  labs(x = "PC axis", y = "Variance", title = "Hell PCA Screeplot")

In [ ]:
# To visualize the PCA, first make a dataframe of the data

carhell_pca_out <- data.frame(carhell_pca$x) %>% 
  rownames_to_column(var="Name") %>%
  separate(Name, into =c("Feature", "Month", "Depth")) %>%
  data.frame
head(carhell_pca_out)

In [ ]:
ggplot(carhell_pca_out) +
  geom_point(aes(x = PC1, y = PC2, fill=Feature), shape =21, size = 8) +
  ylab(paste0('PC2 ', round(carhell_pca_variances[2,2]*100,2),'%')) + #Extract y axis value from variance
  xlab(paste0('PC1 ', round(carhell_pca_variances[1,2]*100,2),'%')) + #Extract x axis value from variance
  scale_fill_brewer(palette = 'Set3', name = 'Oxygen Regime') +  
  ggtitle('Hellinger PCA Ordination') +
  coord_fixed(ratio = 1) +
  theme_bw()+theme(text = element_text(size = 18))

In [ ]:
#fit the environmental factors
#import metadata table
metadata<-read.delim("input_data/Cariaco_metadata.txt")

#The function fits environmental vectors or factors onto an ordination. The projections of points onto vectors have maximum correlation with corresponding environmental variables, and the factors show the averages of factor levels.
envfit(carhell_pca, metadata, na.rm = TRUE, permutations = 9999)

In [ ]:
#covert to dataframe
envfactors<-envfit(carhell_pca, metadata, na.rm = TRUE, permutations = 9999)
en_coord_cont = as.data.frame(scores(envfactors, "vectors")) * ordiArrowMul(envfactors)

#and choose only the significant factors to plot
en_coord_cont%>%rownames_to_column()%>%filter(rowname=="Depth"|rowname =="H2S") %>% column_to_rownames()->en_coord_cont_sign

In [ ]:
ggplot(carhell_pca_out) +
  geom_point(aes(x = PC1, y = PC2, fill = Feature), size = 8, shape = 21, color = "black") +
  geom_segment(data=en_coord_cont_sign,aes(x=0,xend=PC1,y=0,yend=PC2),
      arrow = arrow(length = unit(0.3, "cm")),colour="grey30") + 
  geom_text(data=en_coord_cont_sign,aes(x=PC1,y=PC2, label= row.names(en_coord_cont_sign)),size=3, vjust=-0.5)+
  ylab(paste0('PC2 ', round(carhell_pca_variances[2,2]*100,2),'%')) + #Extract y axis value from variance
  xlab(paste0('PC1 ', round(carhell_pca_variances[1,2]*100,2),'%')) + #Extract x axis value from variance
  scale_fill_brewer(palette = 'Spectral', name = 'Oxygen Regime', limits=c("CarOxic", "CarDysox","CarSuboxic","CarAnox","CarSulf", "CarEux"),   labels=c("Oxic",  "Dysoxic", "Suboxic", "Anoxic", "Sulfidic", "Euxinic")) +
  ggtitle('Hellinger PCA Ordination') +
  coord_fixed() +
  theme_bw()+ theme(text = element_text(size = 18))

In [ ]:
#for the interest of time we worked with a small dataset (*small refers to the total number of samples)
download.file("http://taraoceans.sb-roscoff.fr/EukDiv/data/Database_W5_OTU_occurences.tsv.zip", "/home/jovyan/ohw20-tutorials/02-R-genomicviz/input_data/Database_W5_OTU_occurences.tsv.zip")

unzip("input_data/Database_W5_OTU_occurences.tsv.zip", exdir="input_data")

Tara_euk_OTU <- readr::read_tsv("input_data/Database_W5_OTU_occurences.tsv")

download.file('http://taraoceans.sb-roscoff.fr/EukDiv/data/Database_W1_Sample_parameters.xls', "/home/jovyan/ohw20-tutorials/02-R-genomicviz/input_data/Database_W1_Sample_parameters.xls")

sample_info <- read_excel("input_data/Database_W1_Sample_parameters.xls")

In [ ]:
ls()


## Heatmaps

[Heatmaps](https://jcoliver.github.io/learn-r/006-heatmaps.html) can be used to visualize any type of data, including the OTU (or ASV) tables as the ones we used above. But for this example we will use a dataset from another [study](https://science.sciencemag.org/content/358/6366/1046), where we investigated the biogeography of genomes belonging to the functional group "Nitrite-Oxidizing Bacteria" (important components of the nitrogen and carbon cycles) in the mesopelagic.

In [ ]:
NOB_table<-read.delim("input_data/Tara_summary_table_pctid95_minlen100_overlap0.txt")

In [ ]:
head(NOB_table)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 300)
ggplot(NOB_table, aes(metagenome, sag, fill= prop_total_MG_bp_recruited_per_SAG_mbp)) + 
  geom_tile()

That looks awful for many more reasons that I have time to mention

In [ ]:
#change the scale to logarithmic; change the colors (with reflect zeros; cold color for low abundance; warm color for high adundance )
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 300)
ggplot(NOB_table, aes(metagenome, sag, fill= prop_total_MG_bp_recruited_per_SAG_mbp)) + 
  geom_tile()+scale_fill_gradientn(colours=c("white","lightblue","red"), trans="log", na.value="white",breaks=c(0.00000001, 0.0000001,0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1))+ theme_bw()+theme(legend.position = "bottom",legend.title = element_blank(),  legend.text = element_text(size = 10, angle = 45,hjust = 1, vjust = 1), axis.text.x = element_text(size = 10, angle = 45,hjust = 1, vjust = 1), axis.text.y = element_text(size = 12))

That is a bit better but it still needs impovement; there is a pattern related to the origin of metagenomes

In [ ]:
###write your own code; try to cluster (use Bray-curtis or any other similary index) both metagenomes and metatranscriptomes. Does the heatmap looks better?

### I find that beautifying heatmas is extremely changing. Fortunately Dr. Julia Brown, wrote a script that creates beautiful plots https://github.com/juliambrosman/sag-mg-recruit/blob/master/smr_plotting_functions.R


### Maps (the biologists way i.e. totally primitive)

In [ ]:
#Load one more library for the map constuction
library(maps)
library(mapdata)

In [ ]:
#Clean the abundance table to only keep one genome "AG-538-K21" for this example; ant its abundance in any of the metagenones investigated
AG538K21abund<-NOB_table%>%filter(sag=="AG-538-K21")%>%
  separate(metagenome, c("Expedition", "Station_number", "Depth"), sep = "_", remove = FALSE)%>%
  unite("Station", Expedition:Station_number, remove = FALSE)

AG538K21abund<-AG538K21abund[,c("sag", "Station", "prop_total_MG_bp_recruited_per_SAG_mbp")]

In [ ]:
#Use the sample info table we downloaded from Tara to get the coordinates of each metagenome
sample_info_coord<-sample_info%>%select(Station=2, Lat=9, Lon=10)%>%distinct()%>%
  group_by(Station)%>%filter(row_number()==1)

In [ ]:
#merge the two table
AG538K21abund_coord<-merge(AG538K21abund,sample_info_coord, by.x="Station")

In [ ]:
#I like to make my own colors and this is another way to do it. I need one of them to be transparent
red_custom <- rgb(255, 33, 33, max = 255, alpha = 255, names = "red_custom")
red_custom_t <- rgb(255, 33, 33, max = 255, alpha = 175, names = "red_custom_t")

In [ ]:
map('world',col='gray45', border= "gray45", fill=TRUE)
points(AG538K21abund_coord$Lon, AG538K21abund_coord$Lat, pch=21, col=red_custom, bg=red_custom_t, cex=10*100*2.54*AG538K21abund_coord$prop_total_MG_bp_recruited_per_SAG_mbp)

#scale 0.5%
points(140, -74, pch=21, col=red_custom, bg="white", cex=5)

*Disclaimer: I cannot figure out how to adjust the size of the map in the notebook. But it looks ok exported. Use this sequence of command to export any of the other figures.

In [ ]:
# make an output directory for your plot (if there is not any)
dir.create(output_figures)
# open a pdf file
pdf("output_figures/map.pdf") 
# create the plot
map('world',col='gray45', border= "gray45", fill=TRUE)
points(AG538K21abund_coord$Lon, AG538K21abund_coord$Lat, pch=21, col=red_custom, bg=red_custom_t, cex=10*100*2.54*AG538K21abund_coord$prop_total_MG_bp_recruited_per_SAG_mbp)
# close the pdf file
dev.off() 

I am certain that you are going to learn better ways to plot data on the maps. Consider the abundance of a specific taxon or group as a variable and try plotting it using the tools you will learn. 